**Mohammed errakho - adill oussidi**

In [ ]:
import numpy as np 
import pandas as pd
import os 

import glob
from tqdm.auto import tqdm
import re

In [ ]:
# tenserflow imports
import tensorflow as tf

In [ ]:
# nlp libaries imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
main = "../input/cardiovascular-disease-identification-2nd/"
train = "../input/cardiovascular-disease-identification-2nd/data/data/train/*"
test =  "../input/cardiovascular-disease-identification-2nd/data/data/test/*"


In [ ]:
train1 = pd.read_csv("../input/cardiovascular-disease-identification-2nd/train.csv")
test2 = pd.read_csv("../input/cardiovascular-disease-identification-2nd/test.csv")
submission = pd.read_csv("../input/cardiovascular-disease-identification-2nd/sample_submission.csv")

In [ ]:
train1["text"] = ["" for _ in  range(train1.shape[0])]
test1["text"] = ["" for _ in  range(test1.shape[0])]

In [ ]:
classesPaths = glob.glob(train)

In [ ]:
NUM_CLASSES = len(classesPaths)
def load_train_text(train1):
    docs = [[] for i in range(NUM_CLASSES)]
    for path_id in tqdm(range(NUM_CLASSES)):
        
        path = classesPaths[path_id]
        docPaths = glob.glob(os.path.join(path,'*'))
        for docPath in docPaths:
            with open(docPath, 'r') as f:
                text = f.read()
            filename = int(docPath[71:])
            indice = train1[train1.filename == filename].index
            train1.loc[indice,"text"] = text
            docs[path_id].append(text)
            
    return docs

def load_test_text(test):
    docs = []
    docPaths = glob.glob(testPath)
    
    for docPath in tqdm(docPaths):
        with open(docPath, 'r') as f:
            text = f.read()
        filename = int(docPath[67:])
        indice = test1[test1.filename == filename].index
        test1.loc[indice,"text"] = text
        docs.append(text)
            
    return docs

In [ ]:
load_train_text(train1)
load_test_text(test1)

In [ ]:
# this an helper function for proccessing text for nlp purposes
def process_text(text) : 
    text = text.lower()
    text = re.sub( r'[0-9]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace("\n" ,"")
    text = text.strip()
    stop_words = set(stopwords.words('english'))
    text_tokenizer = word_tokenize(text)
    text = " ".join([w for w in text_tokenizer if  w not in stop_words])
    
    return text 

In [ ]:
train1.text = train1.text.apply(process_text)
test1.text = test1.text.apply(process_text)

In [ ]:
VOCAB_SIZE = 100000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train1["text"])

In [ ]:
X= train1.text
y = train1.iloc[:,1:24]

In [ ]:
vocab = np.array(encoder.get_vocabulary())

In [ ]:
def model():
    model = tf.keras.Sequential([encoder,tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),output_dim=128,mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(23,activation = "softmax")
])
    return model

In [ ]:
model = model()

In [ ]:
model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
history = model.fit(X,y, epochs=20)

In [ ]:
preds = model.predict(test1.text)
preds = np.round(preds,0).astype(int)
filename = list(test1['filename'].values)

In [ ]:
submission = pd.DataFrame(preds, columns=['target_00', 'target_01',	'target_02', 'target_03','target_04','target_05','target_06','target_07', 'target_08', 'target_09','target_10','target_11','target_12','target_13','target_14', 'target_15','target_16', 'target_17', 'target_18', 'target_19', 'target_20','target_21', 'target_22'])

In [ ]:
submission['filename'] = filename
submission.to_csv('submission.csv', index=False)